In [6]:
search_for = 119
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.007142066955566406
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 248563541
type: [1, 1, 1, 1, 119] 119
cases of this type: 200533921
10000 16.829180573414543
20000 17.388069450927734
30000 16.766485942974157
40000 16.618301905159605
50000 59.37164579372434
60000 75.41754548237982
70000 79.00430597153651
80000 65.23084153139293
90000 75.76169157799906
100000 74.70512674031008
110000 81.79048353450125
120000 82.97068951865174
130000 90.58157791648323
140000 109.8890765846227
150000 87.63135532294592
160000 99.20295825910195
170000 99.47732606369343
180000 89.90713227196073
190000 102.70405420322577
200000 89.70805441775899
210000 112.75728146442697
220000 104.6743306969835
230000 100.78156749739938
240000 118.64800567448647
250000 117.37753779374826
260000 149.0110629622878
270000 157.25542864254786
280000 194.83147771136225
290000 187.0206646306882
300000 175.57429713125012
310000 185.37625310

3050000 202.4231758844457
3060000 205.09547881199342
3070000 209.31480651449644
3080000 201.33802086145036
3090000 200.69834416529443
3100000 208.27208859580514
3110000 198.7184915964785
3120000 209.61290439155627
3130000 194.6536744422952
3140000 211.03321709438194
3150000 207.62340922403183
3160000 198.31179726080597
3170000 225.4010286994427
3180000 194.43934432105362
3190000 214.38920276548825
3200000 212.69188370029968
3210000 203.93574853254384
3220000 223.2486826905914
3230000 189.64773124649466
3240000 226.71907214603092
3250000 204.95372896812748
3260000 203.35321968364678
3270000 218.3285783063293
3280000 177.85326662140176
3290000 214.42544406371712
3300000 194.8357464387488
3310000 250.93309178279046
3320000 127.24107264771374
3330000 53.95667578710192
3340000 57.13197099328765
3350000 54.1181020988667
3360000 57.50252111644364
3370000 56.29493959021368
3380000 43.67551782172249
3390000 45.5547059232639
3400000 43.946396502697446
3410000 44.303340552324514
3420000 44.166917

6130000 184.55968733087605
6140000 168.7077142773418
6150000 174.18232295779592
6160000 171.79012798230562
6170000 174.19631809162948
6180000 172.77400574763692
6190000 166.23004017151624
6200000 177.33342785421667
6210000 171.37479856927436
6220000 171.31844484659644
6230000 184.83241420513846
6240000 176.18123895137174
6250000 164.39946938255767
6260000 173.86260618306565
6270000 180.19300384264247
6280000 194.86651559284647
6290000 183.0796741233598
6300000 176.98086572710915
6310000 166.02359844225933
6320000 170.9398297487406
6330000 201.56651618231334
6340000 185.37716880633386
6350000 177.78163369051597
6360000 184.74527506491722
6370000 187.87297822272808
6380000 182.41403717355067
6390000 178.44770442502747
6400000 187.25534058539313
6410000 181.56352399965147
6420000 175.23274096714115
6430000 178.76556222336527
6440000 187.89154221347363
6450000 180.65099431084735
6460000 167.60130712577003
6470000 188.91969157630305
6480000 180.70530341286124
6490000 181.213169733091
650000

9210000 177.12954057610006
9220000 156.1433378268954
9230000 223.02504837029642
9240000 67.50379624775131
9250000 20.81389067297995
9260000 20.947425281913915
9270000 20.809558509162095
9280000 20.784087811080415
9290000 20.815856584413886
9300000 20.859708219986818
9310000 152.8390341820159
9320000 188.68327264223427
9330000 196.38568805720476
9340000 142.86028790201814
9350000 169.2347102392401
9360000 163.68030232500396
9370000 159.52545601599192
9380000 179.2319547233781
9390000 164.65171548859647
9400000 186.25100142337075
9410000 175.718680203192
9420000 173.25515758117052
9430000 186.57757555929558
9440000 165.01264864600088
9450000 189.67070165171222
9460000 175.47923773960943
9470000 171.55244795991715
9480000 174.3344008202742
9490000 162.36775926841185
9500000 174.26422442274438
9510000 160.5976881049367
9520000 167.923754292052
9530000 170.0907569175221
9540000 166.11517770800842
9550000 172.03607623499323
9560000 164.7942966231483
9570000 170.15268074135784
9580000 168.672

12210000 182.87751624031011
12220000 170.13832731441352
12230000 174.27390468280637
12240000 163.61261788590767
12250000 178.66198330616058
12260000 186.42053280155125
12270000 180.20746620190025
12280000 168.1722809053324
12290000 196.65441001990473
12300000 176.28575121554385
12310000 173.96440617424525
12320000 167.34842936066684
12330000 171.46489635953708
12340000 168.21964174382094
12350000 162.78015930654269
12360000 173.73513441007185
12370000 158.9945024883887
12380000 166.65366789854545
12390000 168.53557225115824
12400000 172.87863917397027
12410000 179.92459490066307
12420000 161.71332287600768
12430000 175.18364114252472
12440000 179.14166214697093
12450000 168.8213257396225
12460000 172.9521554949327
12470000 168.5087264038564
12480000 178.31192503797
12490000 166.62888760933322
12500000 166.34031953584446
12510000 172.62433251415135
12520000 155.8371791310588
12530000 180.47750104334733
12540000 158.06689743167527
12550000 172.92231748709946
12560000 164.63558108835855
1

...

235340000 3.043627203585466
235350000 2.692314648999744
235360000 1.6224968756772544
235370000 1.93173669834584
235380000 3.39118169444769
235390000 2.2138344742360383
235400000 1.8778906459743976
235410000 3.0572117791904345
235420000 3.801304620457689
235430000 3.0007934759731296
235440000 3.477438237819321
235450000 3.795571618110087
235460000 3.82784935009253
235470000 3.8989927811680434
235480000 4.275730945425769
235490000 3.3900581034730277
235500000 2.539979578621222
235510000 3.515770257282145
235520000 3.6908096602268814
235530000 2.218282051873015
235540000 2.6612436630090146
235550000 2.9094706607683896
235560000 2.3188399666035306
235570000 1.7033909092262915
235580000 2.947429478403191
235590000 3.0447926942678296
235600000 1.6201909006627135
235610000 1.9920334113452434
235620000 3.5619236493193376
235630000 2.834611214199046
235640000 2.244649453291893
235650000 3.7265588786714674
235660000 4.088396808405141
235670000 3.5009069216471516
235680000 3.3597165372558235
2356

238220000 1.38077949123919
238230000 2.4015716631095545
238240000 1.4005377148397034
238250000 1.2055015163605214
238260000 1.741722694167713
238270000 2.7079280400545795
238280000 1.9522625542166232
238290000 1.8673264991052547
238300000 3.032301863112145
238310000 2.606976549505174
238320000 2.392340048608482
238330000 3.074972110738403
238340000 3.538215663405836
238350000 3.3276516267407934
238360000 3.171579255655375
238370000 3.2802011593103604
238380000 1.5849519502205318
238390000 1.547218858202484
238400000 2.5338694794691006
238410000 1.38999758348861
238420000 1.1124544680665533
238430000 1.6946371964805722
238440000 2.602422530276715
238450000 1.3386299754537212
238460000 1.263441662959695
238470000 2.757051930495547
238480000 2.0693615529840588
238490000 1.0386758958358564
238500000 1.6270590764291353
238510000 2.6770679042575027
238520000 1.4779147733377218
238530000 1.3186142350094185
238540000 3.2608337178327247
238550000 2.983433104706963
238560000 2.684857986500316
23

241060000 0.46493183726255766
241070000 0.5026521971117655
241080000 0.4886087234930727
241090000 0.42281162458869487
241100000 0.48359740089801945
241110000 0.48042123740080334
241120000 0.48143235945549934
241130000 0.48527691910493376
241140000 0.5044526470772095
241150000 0.47720779077402087
241160000 0.43280392539381973
241170000 0.43744679244291784
241180000 0.41511476527984276
241190000 0.4489051222421925
241200000 0.444787984167152
241210000 0.42241950901130165
241220000 0.4516282064856291
241230000 0.48702933455531466
241240000 0.38015727517014086
241250000 0.3084576121788422
241260000 0.4533597547448675
241270000 0.4823084551012384
241280000 0.360890220363458
241290000 0.4085339879864719
241300000 0.46726990636902394
241310000 0.47408257477909327
241320000 0.395398566857742
241330000 0.4867166240823136
241340000 0.45859955965258675
241350000 0.43511477261747916
241360000 0.3701572843673825
241370000 0.38362223422835273
241380000 0.4624325773086415
241390000 0.4618404799935618

243850000 0.9876140853941242
243860000 1.388312622366508
243870000 1.5116033790890508
243880000 1.445391909796324
243890000 1.458248742212991
243900000 1.6901982216379907
243910000 1.4308115404620634
243920000 1.3809680785022378
243930000 1.5804900825190877
243940000 1.3769856684239838
243950000 1.1319255162603854
243960000 1.3717710005009902
243970000 1.5020244565751883
243980000 1.1976982092209856
243990000 1.2843609212260247
244000000 1.407106363796499
244010000 1.3591073565031289
244020000 1.244035407619059
244030000 1.5643938470414414
244040000 1.5039304709435501
244050000 1.1553009043587976
244060000 1.4026179629087714
244070000 1.420995501704037
244080000 1.1950860623539952
244090000 1.2823885324972273
244100000 1.606112301365316
244110000 1.0327874686920047
244120000 0.3176999535477757
244130000 0.33327115789727374
244140000 0.26711488775497017
244150000 0.30333833228279483
244160000 0.2685189861686031
244170000 0.31334963098869056
244180000 0.2681068091804187
244190000 0.33457

246660000 0.5531981646330025
246670000 0.5329606655114558
246680000 0.5063957397937178
246690000 0.5654922083341479
246700000 0.558156709696763
246710000 0.6010199791002869
246720000 0.5453904927613006
246730000 0.5140491686502165
246740000 0.5939574684837858
246750000 0.48542049469061693
best so far: 0
type: [1, 1, 7, 17, 1] 119
cases of this type: 240737
246760000 0.4021971737883025
246770000 0.12060899255269766
246780000 0.10895725050610304
246790000 0.11878098952864936
246800000 0.10570452253788708
246810000 0.11905869011757109
246820000 0.10813274512053861
246830000 0.11403206962297359
246840000 0.11630942273938656
246850000 0.09182692533934117
246860000 0.07649103312369188
246870000 0.0876423915362027
246880000 0.10864607289583816
246890000 0.09385977741587162
246900000 0.09246927099900776
246910000 0.0815601693993012
246920000 0.10272182408287128
246930000 0.10643239477227794
246940000 0.10562664140200616
246950000 0.10058522000108162
246960000 0.09218818452978797
246970000 0.07